In [1]:
import os
%pwd


'c:\\Users\\LENOVO\\Downloads\\wine_quality\\End_to_End_Wine_Quality\\research'

In [2]:
os.chdir('../')

In [3]:
%pwd

'c:\\Users\\LENOVO\\Downloads\\wine_quality\\End_to_End_Wine_Quality'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    min_samples_split: int
    min_samples_leaf: int
    target_column: str

In [5]:
from Wine_Quality.constants import *
from Wine_Quality.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_samples_split = params.min_samples_split,
            min_samples_leaf = params.min_samples_leaf,
            target_column = schema.name
            
        )

        return model_trainer_config

In [7]:
import pandas as pd
import os
from Wine_Quality import logger
from sklearn.ensemble import RandomForestRegressor
import joblib

In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        rf = RandomForestRegressor(
            n_estimators=self.config.n_estimators,
            min_samples_split=self.config.min_samples_split,
            min_samples_leaf=self.config.min_samples_leaf,
            random_state=42
        )

        rf.fit(train_x, train_y.values.ravel())

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))



In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-28 15:18:21,560: INFO: common: yaml file: C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\config\config.yaml loaded successfully]
[2024-12-28 15:18:21,568: INFO: common: yaml file: C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\params.yaml loaded successfully]
[2024-12-28 15:18:21,570: INFO: common: yaml file: C:\Users\LENOVO\Downloads\wine_quality\End_to_End_Wine_Quality\schema.yaml loaded successfully]
[2024-12-28 15:18:21,573: INFO: common: created directory at: artifacts]
[2024-12-28 15:18:21,573: INFO: common: created directory at: artifacts/model_trainer]
